In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from PIL import Image
import cv2

import time
import os
from tqdm.notebook import tqdm
from tensorflow import keras
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout 
from keras.layers import Conv2DTranspose
from keras.layers import concatenate
import tensorflow as tf
from sklearn.model_selection import KFold

In [4]:
# Directory path of image and mask dataset
IMAGE_PATH = 'drone/dataset/semantic_drone_dataset/original_images/'
MASK_PATH = 'drone/dataset/semantic_drone_dataset/label_images_semantic/'

# 23 classes
NUM_CLASSES = 23 
IMAGE_SIZE = 256

In [5]:
# Create dataframe of image dataset
def create_df():
    name = []
    for dirname, _, filenames in os.walk(IMAGE_PATH):
        for filename in filenames:
            name.append(filename.split('.')[0])
    
    return pd.DataFrame({'id': name}, index = np.arange(0, len(name)))

In [6]:
# Data preprocessing
def preprocess_data(X, y, size):
    X_processed = []
    y_processed = []
    for i in tqdm(range(len(X))):
        img = cv2.imread(IMAGE_PATH + X[i] + '.jpg')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(MASK_PATH + y[i] + '.png', 0)
        img = cv2.resize(img, size)
        mask = cv2.resize(mask, size, interpolation=cv2.INTER_NEAREST)
        X_processed.append(img.reshape(-1, 256, 256, 3))
        y_processed.append(mask.reshape(-1, 256, 256, 1))
    X_processed = np.concatenate(X_processed, axis=0)
    y_processed = np.concatenate(y_processed, axis=0)
    return X_processed, y_processed

df = create_df()
print('Total Images : ', len(df))

Total Images :  400


In [7]:
# Split data into train & test set
# X_trainval, X_test = train_test_split(df['id'].values, test_size=0.3, random_state=19)
# X_train, X_val = train_test_split(X_trainval, test_size=0.15, random_state=19)

# print('Train Size   : ', len(X_train))
# print('Val Size     : ', len(X_val))
# print('Test Size    : ', len(X_test))

In [8]:
# define the desired size of the resized images
size = (256, 256)

# Preprocess the data
X_processed, y_processed = preprocess_data(df['id'].values, df['id'].values, size)
# X_val_processed, y_val_processed = preprocess_data(X_val, X_val, size)
# X_test_processed, y_test_processed = preprocess_data(X_test, X_test, size)

  0%|          | 0/400 [00:00<?, ?it/s]

In [9]:
# Check the shape of X_processed & y_processed
print(X_processed.shape)
print(y_processed.shape)

(400, 256, 256, 3)
(400, 256, 256, 1)


In [10]:
# perform semantic segmentation on one of the images in the dataset
def predictMask(image_id):
    # image_id = '513'
    img = cv2.imread(IMAGE_PATH + image_id + '.jpg')
    # img = cv2.imread("btest2.jpg")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    mask = cv2.imread(MASK_PATH + image_id + '.png', 0)

    img_resized = cv2.resize(img, size)
    mask_resized = cv2.resize(mask, size, interpolation=cv2.INTER_NEAREST)

    img_processed = img_resized.reshape(-1, 3)
    
    return img_processed

In [11]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report,ConfusionMatrixDisplay, roc_curve
from keras.metrics import MeanIoU
from keras.utils import to_categorical
from keras import backend as K
import statistics
import seaborn as sns

def evaluateMask(true_mask, predicted_mask): 
    # Flatten the masks to 1D arrays
    true_mask = to_categorical(true_mask.flatten(), num_classes=23)
    predicted_mask = to_categorical(predicted_mask.flatten(), num_classes=23)
    
    # Mean Iou Score
    mean_iou = MeanIoU(num_classes=23)
    mean_iou.update_state(true_mask, predicted_mask)
    iou_score = mean_iou.result().numpy()
    
    # Confusion matrix
    cm = confusion_matrix(np.argmax(true_mask, axis=1), np.argmax(predicted_mask, axis=1), labels=range(23))
    
    # Calculate the metrics
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    tn = np.sum(cm) - (tp + fp + fn)

    sns.heatmap(cm, annot=True)

    precision = tp / (tp + fp)
    precision_mean = np.mean(np.nan_to_num(precision))
    recall = tp / (tp + fn)
    recall_mean = np.mean(np.nan_to_num(recall))
    accuracy = np.sum(tp) / np.sum(cm)
    f1_score = 2 * precision * recall / (precision + recall)
    f1_score_mean = np.mean(np.nan_to_num(f1_score))
    dice_coefficient = (2 * tp) / (2 * tp + fp + fn)
    dice_coefficient_mean = np.mean(np.nan_to_num(dice_coefficient))
    
    # Print the results
    print("Classification Report:\n", classification_report(np.argmax(true_mask, axis=1), np.argmax(predicted_mask, axis=1)))
    print("Mean IoU score:", iou_score)
    print("Recall:", recall_mean)
    print("Precision:", precision_mean)
    print("Accuracy:", accuracy)
    print("F1 score:", f1_score_mean)
    print("Dice coefficient:", dice_coefficient_mean)

    # Plot the confusion matrix
    sns.heatmap(cm, xticklabels=['P0', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7'
                                 , 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14'
                                 , 'P15', 'P16', 'P17', 'P18', 'P19', 'P20', 
                                 'P21', 'P22'], 
                yticklabels=['A0', 'A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7'
                            , 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15'
                            , 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22'],
    annot=True, fmt='d', annot_kws={'fontsize':6}, cmap="YlGnBu")

In [12]:
def conv_block(inputs=None, n_filters=32, dropout_prob=0, max_pooling=True):
    
    conv = Conv2D(n_filters, 
                  3,      
                  activation='relu',
                  padding='same',
                  kernel_initializer='he_normal')(inputs)
    conv = Conv2D(n_filters, 
                  3,   
                  activation='relu',
                  padding='same',
                  kernel_initializer='he_normal')(conv)
    
    if dropout_prob > 0:
        conv = Dropout(dropout_prob)(conv)      
    
    if max_pooling:
        next_layer = MaxPooling2D(2,strides=2)(conv)
        
    else:
        next_layer = conv
        
    skip_connection = conv
    
    return next_layer, skip_connection

In [13]:
def summary(model):
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    result = []
    for layer in model.layers:
        descriptors = [layer.__class__.__name__, layer.output_shape, layer.count_params()]
        if (type(layer) == Conv2D):
            descriptors.append(layer.padding)
            descriptors.append(layer.activation.__name__)
            descriptors.append(layer.kernel_initializer.__class__.__name__)
        if (type(layer) == MaxPooling2D):
            descriptors.append(layer.pool_size)
        if (type(layer) == Dropout):
            descriptors.append(layer.rate)
        result.append(descriptors)
    return result

In [14]:
input_size=(256, 256, 3)
n_filters = 32
inputs = Input(input_size)
cblock1 = conv_block(inputs, n_filters * 1)
model1 = tf.keras.Model(inputs=inputs, outputs=cblock1)

output1 = [['InputLayer', [(None, 256, 256, 3)], 0],
            ['Conv2D', (None, 256, 256, 32), 896, 'same', 'relu', 'HeNormal'],
            ['Conv2D', (None, 256, 256, 32), 9248, 'same', 'relu', 'HeNormal'],
            ['MaxPooling2D', (None, 128, 128, 32), 0, (2, 2)]]

print('Block 1:')
for layer in summary(model1):
    print(layer)

#comparator(summary(model1), output1)

inputs = Input(input_size)
cblock1 = conv_block(inputs, n_filters * 32, dropout_prob=0.1, max_pooling=True)
model2 = tf.keras.Model(inputs=inputs, outputs=cblock1)

output2 = [['InputLayer', [(None, 256, 256, 3)], 0],
            ['Conv2D', (None, 256, 256, 1024), 28672, 'same', 'relu', 'HeNormal'],
            ['Conv2D', (None, 256, 256, 1024), 9438208, 'same', 'relu', 'HeNormal'],
            ['Dropout', (None, 256, 256, 1024), 0, 0.1],
            ['MaxPooling2D', (None, 128, 128, 1024), 0, (2, 2)]]
           
print('\nBlock 2:')   
for layer in summary(model2):
    print(layer)
    
#comparator(summary(model2), output2)

Block 1:
['InputLayer', [(None, 256, 256, 3)], 0]
['Conv2D', (None, 256, 256, 32), 896, 'same', 'relu', 'HeNormal']
['Conv2D', (None, 256, 256, 32), 9248, 'same', 'relu', 'HeNormal']
['MaxPooling2D', (None, 128, 128, 32), 0, (2, 2)]

Block 2:
['InputLayer', [(None, 256, 256, 3)], 0]
['Conv2D', (None, 256, 256, 1024), 28672, 'same', 'relu', 'HeNormal']
['Conv2D', (None, 256, 256, 1024), 9438208, 'same', 'relu', 'HeNormal']
['Dropout', (None, 256, 256, 1024), 0, 0.1]
['MaxPooling2D', (None, 128, 128, 1024), 0, (2, 2)]


In [15]:
def upsampling_block(expansive_input, contractive_input, n_filters=32):
    
    up = Conv2DTranspose(
                 n_filters,    
                 3,   
                 strides=2,
                 padding='same')(expansive_input)

    merge = concatenate([up, contractive_input], axis=3)
    conv = Conv2D(n_filters,   
                 3,     
                 activation='relu',
                 padding='same',
                 kernel_initializer='he_normal')(merge)
    conv = Conv2D(n_filters,  
                 3,   
                 activation='relu',
                 padding='same',
                 kernel_initializer='he_normal')(conv)
    
    return conv

In [16]:
input_size1=(32, 32, 256)
input_size2 = (64, 64, 128)
n_filters = 32
expansive_inputs = Input(input_size1)
contractive_inputs =  Input(input_size2)
cblock1 = upsampling_block(expansive_inputs, contractive_inputs, n_filters * 1)
model1 = tf.keras.Model(inputs=[expansive_inputs, contractive_inputs], outputs=cblock1)

output1 = [['InputLayer', [(None, 32, 32, 256)], 0],
            ['Conv2DTranspose', (None, 64, 64, 32), 73760],
            ['InputLayer', [(None, 64, 64, 128)], 0],
            ['Concatenate', (None, 64, 64, 160), 0],
            ['Conv2D', (None, 64, 64, 32), 46112, 'same', 'relu', 'HeNormal'],
            ['Conv2D', (None, 64, 64, 32), 9248, 'same', 'relu', 'HeNormal']]

print('Block 1:')
for layer in summary(model1):
    print(layer)

#comparator(summary(model1), output1)

Block 1:
['InputLayer', [(None, 32, 32, 256)], 0]
['Conv2DTranspose', (None, 64, 64, 32), 73760]
['InputLayer', [(None, 64, 64, 128)], 0]
['Concatenate', (None, 64, 64, 160), 0]
['Conv2D', (None, 64, 64, 32), 46112, 'same', 'relu', 'HeNormal']
['Conv2D', (None, 64, 64, 32), 9248, 'same', 'relu', 'HeNormal']


In [17]:
def unet_model(input_size=(256, 256, 3), n_filters=32, n_classes=23):
    inputs = Input(input_size)
    
    cblock1 = conv_block(inputs, n_filters)
    
    cblock2 = conv_block(cblock1[0], n_filters*2)
    cblock3 = conv_block(cblock2[0], n_filters*4)
    cblock4 = conv_block(cblock3[0], n_filters*8, dropout_prob=0.3) 
    cblock5 = conv_block(cblock4[0], n_filters*16, dropout_prob=0.3, max_pooling=None) 
    
    ublock6 = upsampling_block(cblock5[0],cblock4[1] ,  n_filters * 8)
     
    ublock7 = upsampling_block(ublock6, cblock3[1],  n_filters*4)
    ublock8 = upsampling_block(ublock7, cblock2[1],  n_filters*2)
    ublock9 = upsampling_block(ublock8, cblock1[1],  n_filters*1)

    conv9 = Conv2D(n_filters,
                 3,
                 activation='relu',
                 padding='same',
                 kernel_initializer='he_normal')(ublock9)

    conv10 = Conv2D(n_classes, 1, padding='same')(conv9)
    
    model = tf.keras.Model(inputs=inputs, outputs=conv10)

    return model

In [18]:
unet_model_output = [['InputLayer', [(None, 256, 256, 3)], 0],
['Conv2D', (None, 256, 256, 32), 896, 'same', 'relu', 'HeNormal'],
['Conv2D', (None, 256, 256, 32), 9248, 'same', 'relu', 'HeNormal'],
['MaxPooling2D', (None, 128, 128, 32), 0, (2, 2)],
['Conv2D', (None, 128, 128, 64), 18496, 'same', 'relu', 'HeNormal'],
['Conv2D', (None, 128, 128, 64), 36928, 'same', 'relu', 'HeNormal'],
['MaxPooling2D', (None, 64, 64, 64), 0, (2, 2)],
['Conv2D', (None, 64, 64, 128), 73856, 'same', 'relu', 'HeNormal'],
['Conv2D', (None, 64, 64, 128), 147584, 'same', 'relu', 'HeNormal'],
['MaxPooling2D', (None, 32, 32, 128), 0, (2, 2)],
['Conv2D', (None, 32, 32, 256), 295168, 'same', 'relu', 'HeNormal'],
['Conv2D', (None, 32, 32, 256), 590080, 'same', 'relu', 'HeNormal'],
['Dropout', (None, 32, 32, 256), 0, 0.3],
['MaxPooling2D', (None, 16, 16, 256), 0, (2, 2)],
['Conv2D', (None, 16, 16, 512), 1180160, 'same', 'relu', 'HeNormal'],
['Conv2D', (None, 16, 16, 512), 2359808, 'same', 'relu', 'HeNormal'],
['Dropout', (None, 16, 16, 512), 0, 0.3],
['Conv2DTranspose', (None, 32, 32, 256), 1179904],
['Concatenate', (None, 32, 32, 512), 0],
['Conv2D', (None, 32, 32, 256), 1179904, 'same', 'relu', 'HeNormal'],
['Conv2D', (None, 32, 32, 256), 590080, 'same', 'relu', 'HeNormal'],
['Conv2DTranspose', (None, 64, 64, 128), 295040],
['Concatenate', (None, 64, 64, 256), 0],
['Conv2D', (None, 64, 64, 128), 295040, 'same', 'relu', 'HeNormal'],
['Conv2D', (None, 64, 64, 128), 147584, 'same', 'relu', 'HeNormal'],
['Conv2DTranspose', (None, 128, 128, 64), 73792],
['Concatenate', (None, 128, 128, 128), 0],
['Conv2D', (None, 128, 128, 64), 73792, 'same', 'relu', 'HeNormal'],
['Conv2D', (None, 128, 128, 64), 36928, 'same', 'relu', 'HeNormal'],
['Conv2DTranspose', (None, 256, 256, 32), 18464],
['Concatenate', (None, 256, 256, 64), 0],
['Conv2D', (None, 256, 256, 32), 18464, 'same', 'relu', 'HeNormal'],
['Conv2D', (None, 256, 256, 32), 9248, 'same', 'relu', 'HeNormal'],
['Conv2D', (None, 256, 256, 32), 9248, 'same', 'relu', 'HeNormal'],
['Conv2D', (None, 256, 256, 23), 759, 'same', 'linear', 'GlorotUniform']]

In [19]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights = True)

In [20]:
def create_model():
    return unet_model((IMAGE_SIZE, IMAGE_SIZE, 3))

In [24]:
BATCH_SIZE = 8
EPOCHS = 200
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def cross_validation(num_folds):
    kf = KFold(n_splits=num_folds)
    cv_scores, model_history = list(), list()

    for train, test in kf.split(X_processed, y_processed):
        cv_model = None
        cv_model = create_model()
        cv_model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss=loss,
            metrics=["accuracy"],
        )

        X_train, X_test = X_processed[train], X_processed[test]
        y_train, y_test = y_processed[train], y_processed[test]

        train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
        validation_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
        train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=False)
        validation_dataset = validation_dataset.batch(BATCH_SIZE, drop_remainder=False)
        cv_model.save_weights('cv_unet.h5')
        history = cv_model.fit(train_dataset,validation_data=validation_dataset, epochs=EPOCHS, batch_size=8, callbacks=[es])
        _, val_acc = cv_model.evaluate(validation_dataset, verbose = 1)
        cv_model.load_weights('cv_unet.h5')
        print('>%.3f' % val_acc)
        cv_scores.append(val_acc)
        model_history.append(cv_model)

    return np.mean(cv_scores), np.std(cv_scores)

In [25]:
mean, std = cross_validation(5)
print('\nCross Validation with {num_folds} fold')
print('---------------------------------')
print('Estimated Accuracy %.4f (%.6f)' % (mean, std))

Epoch 1/200
40/40 [==============================] - 14s 257ms/step - loss: 14.0565 - accuracy: 0.2796 - val_loss: 2.5501 - val_accuracy: 0.4091
Epoch 2/200
40/40 [==============================] - 9s 232ms/step - loss: 2.3998 - accuracy: 0.4307 - val_loss: 2.2056 - val_accuracy: 0.4847
Epoch 3/200
40/40 [==============================] - 9s 233ms/step - loss: 2.1743 - accuracy: 0.4909 - val_loss: 2.0682 - val_accuracy: 0.5152
Epoch 4/200
40/40 [==============================] - 9s 234ms/step - loss: 2.0481 - accuracy: 0.5164 - val_loss: 1.9917 - val_accuracy: 0.5311
Epoch 5/200
40/40 [==============================] - 8s 205ms/step - loss: 2.0006 - accuracy: 0.5138 - val_loss: 1.8920 - val_accuracy: 0.5382
Epoch 6/200
40/40 [==============================] - 8s 203ms/step - loss: 1.8880 - accuracy: 0.5218 - val_loss: 1.7963 - val_accuracy: 0.5434
Epoch 7/200
40/40 [==============================] - 8s 203ms/step - loss: 1.8254 - accuracy: 0.5302 - val_loss: 1.7335 - val_accuracy: 0.54